In [3]:
import numpy as np
import csv
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Activation, Flatten, MaxPooling2D
from keras.optimizers import SGD

VARIABLE INITIALIZE

In [48]:
SET50 = ['ADVANC', 'AOT', 'BANPU', 'BBL', 'BCP', 'BDMS',
        'BEM', 'BH', 'BJC', 'BTS', 'CENTEL', 'CPALL', 'CPF',
        'CPN', 'DTAC', 'EGCO', 'GLOBAL', 'HMPRO', 'INTUCH', 
        'IRPC', 'KBANK', 'KCE', 'KKP', 'KTB', 'LH', 'MINT', 
        'PTT', 'PTTEP', 'ROBINS', 'SCB', 'SCC', 
        'TCAP', 'TISCO', 'TMB', 'TOP', 'TRUE', 'TU']

In [49]:
lr = 0.95
decay = 1e-6
epochs = 1000
model_size = 9
optimizer = SGD(lr=lr, decay=decay)
activator = 'tanh'
loss_method = 'mse'
metrics = ['mae', 'accuracy']

In [50]:
def load_stock_data(symbol, year_start, year_end):
    stock_data = []
    with open('./SET50/' + symbol + '.BK.csv', 'r') as csv_file:
        file_data = csv.reader(csv_file, delimiter=',')
        file_data = list(file_data)[1:]
        temp_value = 0
        for row in file_data:
            if row[1] is '':
                continue
            elif int(row[0][0:4]) >= year_start and int(row[0][0:4]) <= year_end:
                temp = (float(row[1]) + float(row[4]))/2.0
                if temp_value != 0:
                    unrealize = (temp - temp_value)/ temp_value
                else:
                    unrealize = 0
                unrealize = ["{0:.4f}".format(unrealize)]
                stock_data.append(unrealize)
                temp_value = temp
                
    return stock_data

In [51]:
stock_datas = [ load_stock_data(x, 2007, 2017) for x in SET50 ]

min_count = len(stock_datas[0])

for index, element in enumerate(stock_datas):
    if len(element) < min_count:
        min_count = len(element)

min_count

2043

In [63]:
def create_model(conv_layer, fc_layer):
    model = Sequential()
    for i in range(conv_layer + 1):
        if i == 0:
            model.add(Conv2D(64, (7, 1), input_shape=(len(stock_datas), 30, 1), padding='same'))
        else:
            model.add(Conv2D(32, (3, 1)))
        
        model.add(Activation(activator))
        model.add(MaxPooling2D(pool_size = (2, 1)))
    
    model.add(Flatten())
    
    for i in range(fc_layer + 1):
        if i + 1 == fc_layer:
            model.add(Dropout(0.2))
            model.add(Dense(len(stock_datas)))
        else:
            model.add(Dense(256))  
            model.add(Activation(activator))
            
    return model

In [64]:
models = [create_model(int(i/3), i%3) for i in range(model_size)]

In [65]:
for model in models:
    model.compile(loss=loss_method, optimizer=optimizer, metrics=metrics)

In [66]:
period = 30
rounds = min_count - period

train_index = int(rounds * 0.85)
validate_index = train_index + int(rounds * 0.05)

In [67]:
data = []

for i in range(rounds):
    temp = [i[index : index + 30] for i in stock_datas]
    data.append(temp)
    
data = np.asarray(data)
x_train = data[0: train_index].astype('float32')
x_validate = data[train_index: validate_index].astype('float32')
x_test = data[validate_index: validate_index + 1].astype('float32') # only one month to predict long term

print(x_train.shape, x_validate.shape, x_test.shape)

(1711, 37, 30, 1) (100, 37, 30, 1) (1, 37, 30, 1)


In [68]:
target = []

for i in range(rounds):
    temp = [i[index + 30][0] for i in stock_datas]
    target.append(temp)
    
target = np.asarray(target)
y_train = target[0: train_index].astype('float32')
y_validate = target[train_index: validate_index].astype('float32')
y_test = target[validate_index: rounds].astype('float32')

print(y_train.shape, y_validate.shape, y_test.shape)

(1711, 37) (100, 37) (202, 37)


In [79]:
for (index, model) in enumerate(models):
    model_name = "cnn", str(int(index/3) + 1) + "fc", str(int(index%3) + 1) + ".h5"
    print("cnn:", str(int(index/3) + 1) , "layers, fc:", str(int(index%3) + 1), "layers")
    model.fit(x_train, y_train, validation_data = (x_validate, y_validate), epochs = epochs, batch_size = 1, verbose= 1)
    model.save(model_name)

cnn: 1 layers, fc: 1 layers
cnn: 1 layers, fc: 2 layers
cnn: 1 layers, fc: 3 layers
cnn: 2 layers, fc: 1 layers
cnn: 2 layers, fc: 2 layers
cnn: 2 layers, fc: 3 layers
cnn: 3 layers, fc: 1 layers
cnn: 3 layers, fc: 2 layers
cnn: 3 layers, fc: 3 layers
